In [1]:
import pandas as pd
from WindPy import w
from datetime import datetime, timedelta

# 启动 Wind API
if not w.isconnected():
    start_res = w.start()
    if start_res.ErrorCode != 0:
        print(f"Wind 启动失败: {start_res}")
    else:
        print("Wind 启动成功")

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
Wind 启动成功


In [2]:
# 待测试的映射字典
# Key: 您目前使用的 Wind 指标代码
# Value: 您的目标系统字段名

# 1. 行情序列字段 (通常用于 wsd)
market_map_test = {
    "OPEN": "open",
    "HIGH": "high",
    "LOW": "low",
    "CLOSE": "close",
    "VOLUME": "volume",
    "AMT": "amount",
    "VWAP": "vwap",
    "ADJFACTOR": "adjfactor",
    "PCT_CHG": "pct_chg"
}

# 2. 财务/截面字段 (通常用于 wss)
finance_map_test = {
    "NET_PROFIT_IS": "net_profit",      # 归母净利润
    "TOT_ASSETS": "total_assets",       # 总资产
    "TOT_EQUITY": "total_equity",       # 股东权益
    "S_VAL_MV": "total_mv",             # 总市值
    "DQ_MV": "float_mv"                 # 流通市值
}

print("映射表加载完成")

映射表加载完成


In [3]:
def verify_wind_fields(api_type, codes, field_map, **kwargs):
    """
    api_type: 'wsd' 或 'wss'
    codes: 测试标的（建议选一个流动性好、上市久的标的，如 '000001.SZ'）
    field_map: 待测试的字典 {wind_key: user_key}
    kwargs: 传递给 API 的时间参数等
    """
    
    # 提取 Wind 指标 Key 并转换为逗号分隔字符串
    wind_fields_input = list(field_map.keys())
    fields_str = ",".join(wind_fields_input)
    
    print(f"正在测试 {api_type.upper()} 字段: {fields_str} ...")
    
    # 根据类型调用 API
    if api_type == 'wsd':
        # 测试最近 5 个交易日，确保有数据
        end_date = datetime.now().strftime("%Y-%m-%d")
        start_date = (datetime.now() - timedelta(days=10)).strftime("%Y-%m-%d")
        res = w.wsd(codes, fields_str, start_date, end_date, kwargs.get('options', ""))
    elif api_type == 'wss':
        # 截面数据使用最近交易日
        res = w.wss(codes, fields_str, kwargs.get('options', ""))
    
    # --- 结果分析 ---
    
    if res.ErrorCode != 0:
        print(f"❌ API 请求失败, ErrorCode: {res.ErrorCode}")
        print(f"错误信息: {res.Data}")
        return None

    # 获取 Wind 实际返回的字段列表 (Wind 通常会把合法的字段全部大写返回)
    returned_fields = [f.upper() for f in res.Fields]
    
    # 构建验证报告
    df_report = pd.DataFrame(index=wind_fields_input, columns=['Is_Valid', 'Returned_Name', 'Has_Data', 'Sample_Value'])
    
    # 检查数据是否全为 NaN (取第一行数据作为样本)
    # 注意：res.Data 是 list of list，按字段顺序排列
    
    for i, input_field in enumerate(wind_fields_input):
        input_upper = input_field.upper()
        
        # 1. 检查 Wind 是否返回了该字段
        if input_upper in returned_fields:
            idx_in_res = returned_fields.index(input_upper)
            sample_data = res.Data[idx_in_res]
            
            # 检查样本值
            first_valid_val = next((x for x in sample_data if x is not None and str(x) != 'nan'), None)
            
            df_report.loc[input_field, 'Is_Valid'] = True
            df_report.loc[input_field, 'Returned_Name'] = input_upper
            df_report.loc[input_field, 'Has_Data'] = first_valid_val is not None
            df_report.loc[input_field, 'Sample_Value'] = first_valid_val
        else:
            # 如果输入的字段不在返回列表中，说明 Wind 不识别该代码
            df_report.loc[input_field, 'Is_Valid'] = False
            df_report.loc[input_field, 'Returned_Name'] = "NOT_FOUND"
            df_report.loc[input_field, 'Has_Data'] = False
            df_report.loc[input_field, 'Sample_Value'] = "Invalid Field Code"
            
    return df_report

In [5]:
# 测试标的
test_code = "000001.SZ"

# 1. 测试行情字段 (WSD)
# options="PriceAdj=F" 使用前复权测试，防止复权因子导致空值
print("------- 行情字段测试报告 -------")
report_market = verify_wind_fields('wsd', test_code, market_map_test, options="PriceAdj=F")
display(report_market)



------- 行情字段测试报告 -------
正在测试 WSD 字段: OPEN,HIGH,LOW,CLOSE,VOLUME,AMT,VWAP,ADJFACTOR,PCT_CHG ...


,Is_Valid,Returned_Name,Has_Data,Sample_Value
OPEN,True,OPEN,True,11.61
HIGH,True,HIGH,True,11.61
LOW,True,LOW,True,11.51
CLOSE,True,CLOSE,True,11.52
VOLUME,True,VOLUME,True,74236916.0
AMT,True,AMT,True,857197057.0
VWAP,True,VWAP,True,11.546776
ADJFACTOR,True,ADJFACTOR,True,82.611115
PCT_CHG,True,PCT_CHG,True,-0.860585


In [6]:
# 2. 测试财务字段 (WSS)
# options="tradeDate=20231229" 指定一个具体的年报或交易日期，确保有财务数据
print("\n------- 财务字段测试报告 -------")
report_wss = verify_wind_fields('wss', test_code, finance_map_test, options="tradeDate=20240630") # 使用最近半年报日期
display(report_wss)


------- 财务字段测试报告 -------
正在测试 WSS 字段: NET_PROFIT_IS,TOT_ASSETS,TOT_EQUITY,S_VAL_MV,DQ_MV ...
❌ API 请求失败, ErrorCode: -40522006
错误信息: [['CWSSService: invalid indicators.']]


None

In [7]:
w.menu()